In [1]:
## Query real time data from DingXiangYuan, 
## and keep the latest records every day for each city


import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import numpy as np
import math
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from googletrans import Translator # package used to translate Chinese into English
translator = Translator()

In [2]:
DXYArea = pd.read_csv('../data/DXY_Chinese.csv') # Read Chinese version 
# select column
DXYArea = DXYArea[['date','country','countryCode','province', 'city', 'confirmed', 'suspected', 'cured', 'dead']]

DXYArea.tail(2)  

,date,country,countryCode,province,city,confirmed,suspected,cured,dead
15972,2020-03-24,津巴布韦,ZW,NaN,NaN,3,0,0,1
15973,2020-03-24,纳米比亚,NaN,NaN,NaN,4,0,0,0


In [3]:
def isNaN(num):
    return num != num

## Resoruce for Chinese - English Translation
with open('chineseProvince_to_EN.pkl','rb') as f:
    prov_dict = pkl.load(f)

    
with open('chineseCity_to_EN.pkl','rb') as f:
    city_dict = pkl.load(f)    
        
def translate_to_English(data, prov_dict, city_dict):
    """Translate Chinese in dataset to English
    """        
    data['province'] = data['province'].apply(getProvinceTranslation)
    data['city'] = data['city'].apply(getCityTranslation)
    
    for city in unable_translation: # remove these unable translated data
        data = data[data['city']!=city]
    return data
    
def getProvinceTranslation(name):
    if not isNaN(name):
        return prov_dict[name]
    else: 
        return name

unable_translation = []
def getCityTranslation(name):
    try:
        if not isNaN(name): 
            return city_dict[name]
        else:
            return name
    except:
        unable_translation.append(name)
        #print(name + ' cannot be translated\n')
        return name
    

In [4]:
daily_frm_DXYArea = translate_to_English(DXYArea, prov_dict, city_dict)

daily_frm_DXYArea.head(3)


,date,country,countryCode,province,city,confirmed,suspected,cured,dead
0,2019-12-01,中国,CN,NaN,NaN,1,0,0,0
1,2019-12-01,中国,CN,Hubei Province,NaN,1,0,0,0
2,2019-12-01,中国,CN,Hubei Province,Wuhan,1,0,0,0


In [11]:
len(daily_frm_DXYArea) == len(DXYArea)



False

In [10]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)
